## Predição de valor de aposentadoria

Partindo de um servidor público que recebe abono permanência, deseja-se estimar seu ganho quando se aposentar

In [3]:
import pandas as pd

df = pd.read_csv("../servidores.csv")

Como os nomes não interessam ao modelo, podemos retirá-lo da tabela

In [4]:
df = df.drop('nome', axis=1)
df = df.dropna()

Convertendo a coluna de escolaridade em uma coluna numérica

In [5]:
discrete_esco = []

for esco in df.escolaridade:
    if esco == "SEGUNDO GRAU INCOMPLETO" or esco == "4A. SERIE DO PRIMEIRO GRAU COMPLETA":
        discrete_esco.append(0)
    elif esco == "ALFABETIZADO SEM CURSOS REGULARES" or esco == "ENSINO FUNDAMENTAL INCOMPLETO":
        discrete_esco.append(1)
    elif esco == "ENSINO FUNDAMENTAL" or esco == "ENSINO MEDIO":
        discrete_esco.append(2)
    elif esco == "SUPERIOR INCOMPLETO" or esco == "ENSINO SUPERIOR":
        discrete_esco.append(3)
    else:
        discrete_esco.append(4)

df['escolaridade'] = discrete_esco

Preparando os dados de treinamento e teste

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('valor_aposentadoria', axis=1)
y = df.valor_aposentadoria


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=11)

realizando o treinamento utilizando um modelo de RandomForest para regressão

In [10]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=150, criterion="absolute_error")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

Avaliando o erro médio absoluto do modelo

In [11]:
from sklearn.metrics import mean_absolute_error

MAE = mean_absolute_error(y_test,y_pred)
print(MAE)

3386.083455368421


Avaliando o modelo por meio de amostragem k-fold cross-validation

In [12]:
from sklearn.model_selection import cross_validate

# K-fold cross-validation
results = cross_validate(model, X_train,y_train, cv = 10)

print(f"10-fold Score: {results['test_score'].mean()}")

10-fold Score: 0.2865357646663107
